In [5]:

import os
import json

def find_image_pull_issues(base_directory):
    """
    Find which responses have image pull issues based on the k8sgpt.json file.
    
    Args:
        base_directory (str): The path to the base directory containing response-* directories.
        
    Returns:
        dict: A dictionary where keys are response directories and values are the image pull issues found.
    """
    image_pull_issues = {}

    # Iterate over all directories in the base directory
    for response_dir in os.listdir(base_directory):
        response_path = os.path.join(base_directory, response_dir)
        
        # Check if it's a directory and matches the response-* pattern
        if os.path.isdir(response_path) and response_dir.startswith("response-"):
            k8sgpt_file = os.path.join(response_path, "k8sgpt.json")
            
            # Check if the k8sgpt.json file exists
            if os.path.exists(k8sgpt_file):
                with open(k8sgpt_file, "r") as f:
                    try:
                        k8sgpt_data = json.load(f)
                        
                        # Check for image pull issues in the "results" section
                        if k8sgpt_data["results"]:
                            for result in k8sgpt_data["results"]:
                                for error in result.get("error", []):
                                    if "Back-off pulling image" in error["Text"]:
                                        if response_dir not in image_pull_issues:
                                            image_pull_issues[response_dir] = []
                                        image_pull_issues[response_dir].append({
                                            "kind": result.get("kind", "Unknown"),
                                            "name": result.get("name", "Unknown"),
                                            "error": error["Text"]
                                        })
                    except json.JSONDecodeError:
                        print(f"Error: Failed to parse JSON in {k8sgpt_file}")
    
    return image_pull_issues

In [48]:
# Example usage
base_directory = "./gpt3_5/zero_shot/role_system_prompt"  # Replace with the path to your base directory
issues = find_image_pull_issues(base_directory)

# Print results
for response, details in issues.items():
    print(f"Response: {response}")
    for issue in details:
        print(f"  - Kind: {issue['kind']}, Name: {issue['name']}, Error: {issue['error']}")

In [12]:
# Example usage
base_directory = "./gpt4o/zero_shot/baseline_system_prompt"  # Replace with the path to your base directory
base_directory = "./test_responses"
issues = find_image_pull_issues(base_directory)

# Print results
for response, details in issues.items():
    print(f"Response: {response}")
    for issue in details:
        print(f"  - Kind: {issue['kind']}, Name: {issue['name']}, Error: {issue['error']}")

In [23]:
import os  
import json  
from collections import defaultdict  

def aggregate_test_results(base_dir):  
    """  
    Aggregates results from testing.json files in the given directory structure.  
    
    Args:  
        base_dir (str): The base directory containing the responses (e.g., prompt_1/).  
    
    Returns:  
        dict: A dictionary containing aggregated results and missing file information.  
    """  
    aggregated_results = defaultdict(int)  
    missing_files = []  
    detailed_results = []  

    # Iterate through all response directories  
    for response_dir in os.listdir(base_dir):  
        response_path = os.path.join(base_dir, response_dir)  
        
        # Check if it's a response directory (e.g., response-0, response-1, etc.)  
        if os.path.isdir(response_path) and response_dir.startswith("response-"):  
            testing_file = os.path.join(response_path, "testing.json")  
            
            if os.path.exists(testing_file):  
                # Load the testing.json file  
                with open(testing_file, "r") as f:  
                    try:  
                        data = json.load(f)  
                        
                        # Aggregate results  
                        aggregated_results["total_responses"] += 1  
                        aggregated_results["valid_yaml"] += int(data.get("valid_yaml", False))  
                        aggregated_results["kubeconform"] += int(data.get("kubeconform", False))    
                        aggregated_results["secrets_found"] += int(data.get("secrets_found", False))  
                        aggregated_results["mysql_found"] += int(data.get("mysql_found", False))  
                        aggregated_results["wordpress_found"] += int(data.get("wordpress_found", False))  
                        aggregated_results["deployed_successful"] += int(data.get("deployed_successful", False))  
                        aggregated_results["healthy"] += int(data.get("healthy", False))  
                        
                        # Collect detailed results for debugging or further analysis  
                        detailed_results.append({  
                            "response": response_dir,  
                            "polaris_score": data.get("polaris_score", 0),  
                            "healthy": data.get("healthy", False),  
                            "deploy_errors": data.get("deploy_errors", None),  
                            "health_result": data.get("health_result", [])  
                        })  
                    except json.JSONDecodeError:  
                        print(f"Error decoding JSON in {testing_file}")  
            else:  
                # Record responses missing the testing.json file  
                missing_files.append(response_dir)  
    

    # Return the aggregated results and missing file information  
    return {  
        "aggregated_results": dict(aggregated_results),  
        "missing_files": missing_files,  
        "detailed_results": detailed_results  
    }
    

In [50]:
base_directory = "./gpt3_5/zero_shot/baseline_system_prompt"  # Replace with the actual path to your directory  
results = aggregate_test_results(base_directory)  
    
print(f"Aggregated Results: {base_directory}")  
print(json.dumps(results["aggregated_results"], indent=4))  
    
if results["missing_files"]:  
    print("\nResponses Missing testing.json:")  
    print(results["missing_files"])

base_directory = "./gpt3_5/zero_shot/baseline_system_prompt_detailed"  # Replace with the actual path to your directory  
results = aggregate_test_results(base_directory)  
    
print(f"Aggregated Results: {base_directory}")   
print(json.dumps(results["aggregated_results"], indent=4))  
    
if results["missing_files"]:  
    print("\nResponses Missing testing.json:")  
    print(results["missing_files"])  

base_directory = "./gpt3_5/zero_shot/role_best_system_prompt"  # Replace with the actual path to your directory  
results = aggregate_test_results(base_directory)  
    
print(f"Aggregated Results: {base_directory}")  
print(json.dumps(results["aggregated_results"], indent=4))  
    
if results["missing_files"]:  
    print("\nResponses Missing testing.json:")  
    print(results["missing_files"])  


base_directory = "./gpt3_5/zero_shot/role_best_system_prompt_detailed"  # Replace with the actual path to your directory  
results = aggregate_test_results(base_directory)  
    
print(f"Aggregated Results: {base_directory}")  
print(json.dumps(results["aggregated_results"], indent=4))  
    
if results["missing_files"]:  
    print("\nResponses Missing testing.json:")  
    print(results["missing_files"])  

base_directory = "./gpt3_5/zero_shot/role_system_prompt"  # Replace with the actual path to your directory  
results = aggregate_test_results(base_directory)  
    
print(f"Aggregated Results: {base_directory}")  
print(json.dumps(results["aggregated_results"], indent=4))  
    
if results["missing_files"]:  
    print("\nResponses Missing testing.json:")  
    print(results["missing_files"])  

base_directory = "./gpt3_5/zero_shot/role_system_prompt_detailed"  # Replace with the actual path to your directory  
results = aggregate_test_results(base_directory)  
    
print(f"Aggregated Results: {base_directory}")  
print(json.dumps(results["aggregated_results"], indent=4))  
    
if results["missing_files"]:  
    print("\nResponses Missing testing.json:")  
    print(results["missing_files"])  
#print("\nDetailed Results (for debugging):")  
#print(json.dumps(results["detailed_results"], indent=4))

Aggregated Results: ./gpt3_5/zero_shot/baseline_system_prompt
{
    "total_responses": 50,
    "valid_yaml": 50,
    "kubeconform": 49,
    "secrets_found": 1,
    "mysql_found": 47,
    "wordpress_found": 46,
    "deployed_successful": 40,
    "healthy": 3
}
Error decoding JSON in ./gpt3_5/zero_shot/baseline_system_prompt_detailed/response-14/testing.json
Aggregated Results: ./gpt3_5/zero_shot/baseline_system_prompt_detailed
{
    "total_responses": 49,
    "valid_yaml": 49,
    "kubeconform": 49,
    "secrets_found": 48,
    "mysql_found": 49,
    "wordpress_found": 49,
    "deployed_successful": 43,
    "healthy": 0
}
Error decoding JSON in ./gpt3_5/zero_shot/role_best_system_prompt/response-32/testing.json
Aggregated Results: ./gpt3_5/zero_shot/role_best_system_prompt
{
    "total_responses": 49,
    "valid_yaml": 49,
    "kubeconform": 49,
    "secrets_found": 0,
    "mysql_found": 49,
    "wordpress_found": 49,
    "deployed_successful": 45,
    "healthy": 0
}
Error decoding JSON